In [154]:
import re # We clean text using regex
import csv # To read the csv
from collections import defaultdict # For accumlating values
from gensim import corpora # To create corpus and dictionary for the LDA model
from gensim.models import LdaModel # To use the LDA model
import pandas as pd
import nltk

In [155]:
#Load reviews and classifier
fdsreviews_df=pd.read_csv("menulog.csv")
print("\nLoad reviews and classifier :")
print(fdsreviews_df[:40])

#Seperate Hashtags and titles to lists
review_list = fdsreviews_df["Customer_review"].tolist()


Load reviews and classifier :
       Customer_name                                       Customer_url  \
0          Prbath W.  https://www.productreview.com.au/consumer-prof...   
1                C.T  https://www.productreview.com.au/consumer-prof...   
2                Ben  https://www.productreview.com.au/consumer-prof...   
3                Mat  https://www.productreview.com.au/consumer-prof...   
4                Tai  https://www.productreview.com.au/consumer-prof...   
5              Lynne  https://www.productreview.com.au/consumer-prof...   
6                Jay  https://www.productreview.com.au/consumer-prof...   
7         marchedley  https://www.productreview.com.au/consumer-prof...   
8             Smorgs  https://www.productreview.com.au/consumer-prof...   
9              Jules  https://www.productreview.com.au/consumer-prof...   
10      Farhad irani  https://www.productreview.com.au/consumer-prof...   
11           ShTfood  https://www.productreview.com.au/consumer-prof.

In [156]:
data = pd.DataFrame(review_list)
data.head()

,0
0,Worst service i have ever experienced.....Wait...
1,"2 days in a row, waiting an hour to get my foo..."
2,Initially Menulog was great for deliveries. Ov...
3,I am absolutely disgusted at the nunber of inc...
4,Ordered my food in advance to arrive at 12:00p...


In [157]:
review_list = [re.sub(r'[^\w\s]','',str(item)) for item in review_list]

In [158]:
from nltk.corpus import stopwords # To remove stopwords
stopwords = set(stopwords.words('english'))

In [159]:
texts = [[word for word in document.lower().split() if word not in stopwords] for document in review_list]

In [160]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
         frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [161]:
dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(1392 unique tokens: ['2', 'agreed', 'could', 'day', 'delivered']...)


In [162]:
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]]


In [163]:
NUM_TOPICS = 10 # This is an assumption. 
ldamodel = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)#This might take some time.

In [165]:
topics = ldamodel.show_topics()
for topic in topics:
    print(topic)

(0, '0.026*"food" + 0.025*"order" + 0.024*"menulog" + 0.024*"service" + 0.016*"delivery" + 0.015*"driver" + 0.013*"restaurant" + 0.012*"still" + 0.011*"time" + 0.011*"call"')
(1, '0.037*"order" + 0.024*"menulog" + 0.018*"delivered" + 0.015*"restaurant" + 0.014*"food" + 0.011*"driver" + 0.010*"dont" + 0.010*"ordered" + 0.010*"give" + 0.010*"said"')
(2, '0.034*"order" + 0.026*"time" + 0.019*"delivery" + 0.016*"menulog" + 0.016*"never" + 0.016*"food" + 0.015*"app" + 0.015*"use" + 0.013*"service" + 0.013*"ordered"')
(3, '0.019*"menulog" + 0.018*"driver" + 0.017*"order" + 0.017*"door" + 0.014*"delivered" + 0.013*"service" + 0.013*"delivery" + 0.010*"us" + 0.010*"told" + 0.010*"knock"')
(4, '0.035*"food" + 0.029*"menulog" + 0.026*"delivered" + 0.024*"driver" + 0.017*"restaurant" + 0.016*"never" + 0.015*"service" + 0.013*"delivery" + 0.013*"would" + 0.012*"get"')
(5, '0.019*"order" + 0.019*"food" + 0.015*"delivery" + 0.014*"driver" + 0.014*"service" + 0.014*"menulog" + 0.013*"restaurant" + 0.

In [169]:
word_dict = {};
for i in range(NUM_TOPICS):
    words = ldamodel.show_topic(i, topn = 15)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
pd.DataFrame(word_dict)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,food,order,order,menulog,food,order,food,menulog,driver,food
1,order,menulog,time,driver,menulog,food,never,order,time,order
2,menulog,delivered,delivery,order,delivered,delivery,menulog,voucher,food,refund
3,service,restaurant,menulog,door,driver,driver,order,service,order,still
4,delivery,food,never,delivered,restaurant,service,service,food,menulog,delivery
5,driver,driver,food,service,never,menulog,get,never,delivery,service
6,restaurant,dont,app,delivery,service,restaurant,ordered,driver,called,ordered
7,still,ordered,use,us,delivery,ordered,driver,refund,never,delivered
8,time,give,service,told,would,use,delivered,ordered,said,voucher
9,call,said,ordered,knock,get,waiting,said,would,minutes,call


In [153]:
import pyLDAvis.gensim # To visualise LDA model effectively
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [170]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -6.6544968011644094
